In [47]:
import qib
import numpy as np
from tenpy.models.lattice import Kagome
from tenpy.networks.site import SpinHalfSite

L = 48
lat = Kagome(4, 4, [SpinHalfSite() for _ in range(3)], bc='periodic')

N = lat.N_sites
A = np.zeros((N, N), dtype=int)

J = (1, 1, 1)
h = (0, 0, 0)

perms_1 = [[0, 8, 12, 20, 24, 32, 36, 44, 
            2, 9, 14, 21, 26, 33, 38, 45,
           4, 10, 16, 22, 28, 34, 40, 46,
           6, 11, 18, 23, 30, 35, 42, 47], 
           [8, 12, 20, 24, 32, 36, 44, 0, 
            9, 14, 21, 26, 33, 38, 45, 2, 
            10, 16, 22, 28, 34, 40, 46, 4,
            11, 18, 23, 30, 35, 42, 47, 6
           ]]
perms_2 = [[0, 1, 2, 3, 4, 5, 6, 7,
           12, 13, 14, 15, 16, 17, 18, 19,
          24, 25, 26, 27, 28, 29, 30, 31,
          36, 37, 38, 39, 40, 41, 42, 43], 
            [1, 2, 3, 4, 5, 6, 7, 0,
             13, 14, 15, 16, 17, 18, 19, 12,
             25, 26, 27, 28, 29, 30, 31, 24,
             37, 38, 39, 40, 41, 42, 43, 36
            ]]
perms_3 = [[1, 8, 3, 9, 13, 20, 5, 10, 15, 21, 25, 32, 
            7, 11, 17, 22, 27, 33, 37, 44, 
            19, 23, 29, 34, 39, 45, 31, 35, 41, 46, 43, 47], 
           [8, 1, 9, 13, 20, 3, 10, 15, 21, 25, 32, 5, 11, 17, 22, 27, 
            33, 37, 44, 7, 23, 29, 34, 39, 45, 19, 35, 41, 46, 31, 47, 43]]

for perm in perms_1+perms_2+perms_3:
    for i in range(len(perm)//2):
        A[perm[2*i], perm[2*i+1]] = 1
        A[perm[2*i+1], perm[2*i]] = 1


In [49]:
import numpy as np
import scipy.linalg
import time
import rqcopt as oc
from quimb.tensor.tensor_arbgeom_tebd import LocalHamGen, TEBDGen, edge_coloring

# Pauli and identity
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
I2 = np.eye(2)


def layer_from_flat_perm(perm_row, L):
    """perm_row is a flat list of length L."""
    return [(perm_row[2*j], perm_row[2*j+1]) for j in range(len(perm_row) // 2)]
layers_raw = [
    perms_1[0], perms_1[1],
    perms_2[0], perms_2[1],
    perms_3[0], perms_3[1],
]
perms_for_trotter = [layer_from_flat_perm(row, L) for row in layers_raw]

In [52]:
import numpy as np
import quimb as qu
import quimb.tensor as qtn



def _edges_from_permutations(perms_1, perms_2, perms_3):
    """
    Take the three [src, tgt] permutation pairs and return a sorted list of
    unique undirected edges (i, j) with i < j.
    """
    edge_set = set()

    for perms in (perms_1, perms_2, perms_3):
        src, tgt = perms
        for a, b in zip(src, tgt):
            if a == b:
                continue
            i, j = sorted((a, b))
            edge_set.add((i, j))

    return sorted(edge_set)


def build_triangular_PEPS(bond_dim, phys_dim=2,
                          seed=None, dtype="complex128"):
    edges = _edges_from_permutations(perms_1, perms_2, perms_3)
    tn = qtn.TN_from_edges_rand(
        edges,
        D=bond_dim,
        phys_dim=phys_dim,
        seed=seed,
        dtype=dtype,
    )

    return tn, (perms_1, perms_2, perms_3)



In [54]:
import gc

def trotter(peps, t, L, J, perms, dag=False,
                      max_bond_dim=5, dt=0.1, trotter_order=2):
    # Number of steps
    nsteps = abs(int(np.ceil(t / dt)))
    dt = t / nsteps

    # Suzuki splitting
    if trotter_order > 1:
        sm = oc.SplittingMethod.suzuki(2, int(np.log(trotter_order)/np.log(2)))
        indices, coeffs = sm.indices, sm.coeffs
    else:
        indices, coeffs = [0, 1, 2], [1, 1, 1]
    
    hloc1 = J[0]*np.kron(X, X)
    hloc2 = J[1]*np.kron(Y, Y)
    hloc3 = J[2]*np.kron(Z, Z)
    hlocs = (hloc1, hloc2, hloc3)
    Vlist_start = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(-1j*c*dt*hlocs[i])

    for n in range(nsteps):
        for layer, V in enumerate(Vlist_start):
            i = n*len(Vlist_start)+layer
            for perm in perms:
                
                edges = [(perm[2*j], perm[2*j+1]) for j in range(len(perm)//2)]
                H2 = {edge: V for edge in edges}
                ham = LocalHamGen(H2=H2, H1=None)
                tebd = TEBDGen(peps, ham=ham, D=max_bond_dim)
                tebd.sweep(tau=-1)
                peps = tebd.state

                del tebd, ham
                gc.collect()
    return peps


def ccU(peps, Vlist, perms_extended, control_layers, dagger=False, max_bond_dim=10):
    for i, V in enumerate(Vlist):
        if dagger or i not in control_layers:
            perms = perms_extended[i]
            for perm in perms:
                edges = [(perm[2*j], perm[2*j+1]) for j in range(len(perm)//2)]
                H2 = {edge: scipy.linalg.logm(V) for edge in edges}
                ham = LocalHamGen(H2=H2, H1=None)
                tebd = TEBDGen(peps, ham=ham, D=max_bond_dim)
                tebd.sweep(tau=-1)
                peps = tebd.state

                del tebd, ham
                gc.collect()
    return peps

In [78]:
import h5py

Vlists = {}
for t in [0.125]:
    with h5py.File(f'./results/kagome_000_L12_t{t}_layers5.hdf5') as f:
        Vlists[t]  =  f["Vlist"][:]
Vlist = Vlists[0.125]
perms_extended = [perms_1]  + [perms_2] + [perms_3] + [perms_2]  + [perms_1]

In [79]:
chi_overlap = 20
peps, (p1, p2, p3) = build_triangular_PEPS(1, 2)
peps_copy_norm = peps.copy()
ov_tn = peps_copy_norm.make_overlap(
    peps_copy_norm,
    layer_tags=("KET", "BRA"),
)
overlap_approx = ov_tn.contract_compressed(
    optimize="auto-hq",
    max_bond=chi_overlap,
    cutoff=1e-10,
)
norm = np.sqrt(abs(overlap_approx))
peps = peps/np.abs(norm)

ov_tn = peps.make_overlap(
    peps,
    layer_tags=("KET", "BRA"),
)
overlap_approx = ov_tn.contract_compressed(
    optimize="auto-hq",
    max_bond=chi_overlap,
    cutoff=1e-10,
)
print(np.abs(overlap_approx))

1.0000000000000004


/opt/anaconda3/envs/qc/lib/python3.10/site-packages/quimb/tensor/tensor_core.py:8584: UserWarning: The contraction tree is not a compressed one, this may be very inefficient.
  warnings.warn(


In [87]:
from qiskit.quantum_info import state_fidelity
import quimb.tensor as qtn
import quimb
from quimb.tensor.tensor_2d_tebd import TEBD2D, LocalHam2D


peps_E = peps.copy()
peps_T = peps.copy()
peps_C = peps.copy()

BD = 3
nsteps = 1
peps_E = trotter(peps_E.copy(), t, L,  J, perms_1+perms_2+perms_3,
                     dt=t/nsteps, max_bond_dim=BD, trotter_order=2)
peps_aE = ccU(peps_C.copy(), Vlist, perms_extended, [], dagger=False,
                 max_bond_dim=BD)
peps_T = trotter(peps_T.copy(), t, L,  J, perms_1+perms_2+perms_3,
                     dt=t/nsteps, max_bond_dim=BD, trotter_order=1)

peps_T.compress_all(max_bond=BD)
peps_E.compress_all(max_bond=BD)
peps_aE.compress_all(max_bond=BD)

TensorNetworkGenVector(tensors=48, indices=142)

In [89]:
ov_tn = peps_E.make_overlap(
    peps_aE,
    layer_tags=("KET", "BRA"),
)

chi_overlap = 7  # max internal bond during compressed contraction
overlap_approx = ov_tn.contract_compressed(
    optimize="hyper-compressed",
    max_bond=chi_overlap,
    cutoff=1e-10,
)

np.abs(overlap_approx)

/opt/anaconda3/envs/qc/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:329: UserWarning: Trial error: No module named 'kahypar'. Set `HyperOptimizer` kwarg `on_trial_error='raise'` to raise this error, or `on_trial_error='ignore'` to silence.
  warnings.warn(
/opt/anaconda3/envs/qc/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:329: UserWarning: Trial error: No module named 'kahypar'. Set `HyperOptimizer` kwarg `on_trial_error='raise'` to raise this error, or `on_trial_error='ignore'` to silence.
  warnings.warn(
/opt/anaconda3/envs/qc/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:329: UserWarning: Trial error: No module named 'kahypar'. Set `HyperOptimizer` kwarg `on_trial_error='raise'` to raise this error, or `on_trial_error='ignore'` to silence.
  warnings.warn(
/opt/anaconda3/envs/qc/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:329: UserWarning: Trial error: No module named 'kahypar'. Set `HyperOptimizer` kwarg `on_

0.4848677059553654